In [293]:
import re
import praw
import nltk
import string
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import DBSCAN
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.metrics import jaccard_similarity_score
from scipy.spatial import distance
import numpy as np
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import preprocess_string

In [263]:
nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('words')

# Get words from NLTK
# words = set(nltk.corpus.words.words())
# Get stopwords from NLTK
stop_words = set(stopwords.words('english'))
# Lemmatizer and Stemmer
lemmatizer=WordNetLemmatizer()
# stemmer= PorterStemmer()
# Get punctuation
punct_list = [i for i in string.punctuation]

[nltk_data] Downloading package stopwords to /Users/tyiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [264]:
# Load Google's pre-trained Word2Vec model.
goog_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [265]:
# Reddit login info
reddit = praw.Reddit(client_id = 'W1R0o531tGoOtA',
                     client_secret = '1Xeft_Tf9Vv-TkHSkPAMIUtF5Oc',
                     user_agent = 'NLP Project (by /u/metisproj4)',
                     user_name = 'metisproj4',
                     password = 'Password12345')

In [266]:
# Call praw to grab stuff from Reddit
reddits = ['investing', 'stocks', 'personalfinance', 'options', 
           'economics', 'financialindependence', 'stockmarket',
           'marketing', 'SecurityAnalysis', 'AskReddit', 'askscience', 
           'explainlikeimfive', 'books', 'LifeProTips', 
           'technology', 'history', 'dating_advice', 'writing'
          ]

grabbed_list_title = []
grabbed_list_body = []
grabbed_list_comment1 = []
grabbed_list_comment2 = []
grabbed_list_comment3 = []
grabbed_list_upvote_ratio = []

for r in reddits:
    sub_list_title = []
    sub_list_body = []
#     sub_list_upvote_ratio = []
    
    grabbed = reddit.subreddit(r).hot(limit=None)
    for i in grabbed:
        sub_list_title.append(i.title)
        sub_list_body.append(i.selftext)
#         sub_list_upvote_ratio.append(i.upvote_ratio)
    
    grabbed_list_title.append(sub_list_title)
    grabbed_list_body.append(sub_list_body)
#     grabbed_list_upvote_ratio.append(sub_list_upvote_ratio)

In [379]:
nltk.download('words')
words = set(nltk.corpus.words.words())

def clean_text_lem(doc, words):
    nuked = ' '.join(w for w in nltk.wordpunct_tokenize(doc) if w.lower() in words or w.istitle())
    mid_clean = nuked.strip().lower()
    for i in punct_list:
        mid_clean = mid_clean.replace(i, '')
    mid_clean = re.sub(r'\d+', '', mid_clean)
    cleaned = ''
    for i in mid_clean.split():
        if i not in stop_words:
            cleaned = cleaned + ' ' + lemmatizer.lemmatize(i)
    return cleaned.strip()

# Get a lemmatized Reddit corpus
corpus_lem_title = []
corpus_lem_body = []
pos_list = []

for r_index in range(len(reddits)):
    for row_index in range(len(grabbed_list_title[r_index])):
        corpus_lem_title.append(clean_text_lem(grabbed_list_title[r_index][row_index], words))
        corpus_lem_body.append(clean_text_lem(grabbed_list_body[r_index][row_index], words))
        pos_list.append([r_index, row_index])

[nltk_data] Downloading package words to /Users/tyiu/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [333]:
# Count vectorize and get TF and TF_IDF vectors
cv=CountVectorizer(max_df=0.85)
tf_vector=cv.fit_transform(corpus_lem_body)
tf_feature_names = cv.get_feature_names()
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tf_idf_vector = tfidf_transformer.fit_transform(tf_vector)

In [269]:
def display_topics(model, feature_names, no_top_words):
    topic_list = []
    for topic_idx, topic in enumerate(model.components_):
        topic_list.append(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print('Topic %d:' % (topic_idx))
        print(topic_list[topic_idx])
    return topic_list

In [270]:
# For running LDA (topics not so great)
# no_topics = 15

# lda = LatentDirichletAllocation(n_components=no_topics)
# lda.fit(tf_vector)

# X_fitted_lda = lda.transform(tf_vector)

# no_top_words = 7
# display_topics(lda, tf_feature_names, no_top_words)

In [398]:
# Run NMF (Seems better than LDA)
no_topics = 30

nmf = NMF(n_components=no_topics, alpha=.1, l1_ratio=.5, init='nndsvd', random_state=10)
nmf.fit(tf_idf_vector)

no_top_words = 20
X_fitted_nmf = nmf.transform(tf_idf_vector)

In [399]:
topic_list = display_topics(nmf, tf_feature_names, no_top_words)

Topic 0:
date really girl said guy relationship dating want told went go never ask friend met talking talk back going got
Topic 1:
stock price dividend portfolio trading top bought value invest global idea hold chart selling split resistance investor cap looking analysis
Topic 2:
posting post new see thread promotion rudeness relaxed excessive politics warrant apply self tend rule busy finance please consider top
Topic 3:
work get year job one much money could fire make also way life want need working even around lot help
Topic 4:
story character main plot world protagonist novel reader fantasy write want chapter first scene one idea series make person villain
Topic 5:
option price call put strike spread expiration sell position contract loss profit sold risk premium underlying trading selling close value
Topic 6:
earnings click billion revenue per share quarter chart week report release growth year refinitiv elite consensus last calendar today beat
Topic 7:
credit card score account b

In [322]:
# Convert topics to word2vec representation
topic_vectors = []
for topic in topic_list:
    row_vector = []
    for w in topic.split():
        try:
            row_vector.append(goog_model.get_vector(w))
        except:
            print(w)
    topic_vectors.append(row_vector)

redditors
fstocks
aautomoderator


In [323]:
# Get mean word2vec representation for topics
topic_2_vec = []
for vec_list in topic_vectors:
    vec_sum = np.zeros((300,))
    for topic_vec in vec_list:
        vec_sum = vec_sum + topic_vec
    topic_2_vec.append(vec_sum*(1/len(vec_list)))
    
# goog_model.most_similar_cosmul(positive=preprocess_string(topic_list[0]))[0:3]

In [337]:
topics = []
for i, val in enumerate(topic_2_vec):
    topics.append(goog_model.similar_by_vector(val, topn=3, restrict_vocab=50000))
    #print('Topic' + str(i))
    #print(topics[i])

In [326]:
# Change my reddit corpus for titles to word2vec
error_check = 25
debugging = False

title_2_vec = []
for title in corpus_lem_title:
    vec_sum = np.zeros((300,))
    for w in title.split():
        try:
            if debugging:
                print(goog_model.get_vector(w)[error_check])
            vec_sum = vec_sum + goog_model.get_vector(w)
        except:
            continue
    if debugging:
        print('\n', vec_sum[error_check])
    try:
        title_2_vec.append(vec_sum*(1/len(title.split())))
    except:
        title_2_vec.append(np.zeros((300,)))

In [327]:
# Change my reddit corpus for body text to word2vec
error_check = 25
debugging = False

body_2_vec = []
for body in corpus_lem_body:
    vec_sum = np.zeros((300,))
    for w in body.split():
        try:
            if debugging:
                print(goog_model.get_vector(w)[error_check])
            vec_sum = vec_sum + goog_model.get_vector(w)
        except:
            continue
    if debugging:
        print('\n', vec_sum[error_check])
    try:
        body_2_vec.append(vec_sum*(1/len(title.split())))
    except:
        body_2_vec.append(np.zeros((300,)))

In [ ]:
jaccard_similarity_score

In [400]:
topic_list_clean = ['dating',                  #0 
                    'stock dividends',         #1
                    'submission/post rules',   #2
                    'working and job',         #3
                    'fantasy writing',         #4
                    'options',                 #5
                    'big company earnings',    #6
                    'credit card debt',        #7
                    'marketing careers advice',#8
                    'financial planning',      #9
                    'investment advice',       #10
                    'growth company stocks',   #11
                    'investment advice',       #12
                    'random noise',            #13
                    'writing advice',          #14
                    'portfolio advice',        #15
                    'day trading',             #16
                    'general news',            #17
                    'long term investing',     #18
                    'books',                   #19
                    'Trade War',               #20
                    'stock crash recession',   #21
                    'random noise',            #22
                    'random noise',            #23
                    'buy the dip',             #24
                    'fund and legal',          #25
                    'random ideas',            #26
                    'social network dating',   #27
                    'credit and loans',        #28
                    'stock market news'        #29
                   ]

In [401]:
def get_str_topics(string):
    words = set(nltk.corpus.words.words())
    topic_weights = nmf.transform(cv.transform([clean_text_lem(string, words)]))
    topic_w_df = pd.DataFrame()
    topic_w_df['topic'] = topic_list_clean
    topic_w_df['weights'] = topic_weights.T
    return topic_w_df

In [402]:
get_str_topics(corpus_lem_body[200]).sort_values(by='weights', ascending=False)[0:5]

,topic,weights
13,random noise,0.173243
6,big company earnings,0.031878
5,options,0.030407
11,growth company stocks,0.010521
28,credit and loans,0.007302


In [403]:
grabbed_list_body[0][200]

'All of the popular S&P 500 ETFs like SPY, VOO, and IVV all cost over $250/share. Are there any S&P 500 ETFs or other options out there that are cheaper?'

In [554]:
def get_w2vec(string):
    parsed_str = preprocess_string(string)
    vec_sum = np.zeros((300,))
    for w in parsed_str:
        try:
            vec_sum = vec_sum + goog_model.get_vector(w)
        except:
            continue
    return vec_sum*(1/len(parsed_str))

def get_closest_stuff(string, w2vec_matrix):
    w2vec_string = get_w2vec(test_str)
    index = []
    cos_dist = []
    for i, val in enumerate(w2vec_matrix):
        index.append(i)
        if len(val)==0:
            cos_dist.append(1000)
        else:
            cos_dist.append(distance.cosine(w2vec_string, val))
    cos_dist_df = pd.DataFrame()
    cos_dist_df['index'] = index
    cos_dist_df['cos_dist'] = cos_dist
    return cos_dist_df

def get_closest_stuff_detrended(detrended, w2vec_matrix):
    w2vec_string = detrended
    index = []
    cos_dist = []
    for i, val in enumerate(w2vec_matrix):
        index.append(i)
        if len(val)==0:
            cos_dist.append(1000)
        else:
            cos_dist.append(distance.cosine(w2vec_string, val))
    cos_dist_df = pd.DataFrame()
    cos_dist_df['index'] = index
    cos_dist_df['cos_dist'] = cos_dist
    return cos_dist_df

In [568]:
# Get Sentiment of everything

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

sentiment_title = []
sentiment_body = []
for val in pos_list:
    vs_title = analyzer.polarity_scores(grabbed_list_title[val[0]][val[1]])
    sentiment_title.append(vs_title)
    vs_body = analyzer.polarity_scores(grabbed_list_body[val[0]][val[1]])
    sentiment_body.append(vs_body)

In [555]:
def print_posts(df, raw_list, num=10):
    sorted_df = pd.DataFrame()
    sorted_df = df.sort_values(by='cos_dist', ascending=True)[0:num].copy()
    for i, val in enumerate(sorted_df['index']):
        print(raw_list[pos_list[val][0]][pos_list[val][1]])

In [569]:
# input_str = 'I hate stocks'
# cos_dist_df = get_closest_stuff(input_str, title_2_vec)
# print_posts(cos_dist_df, grabbed_list_title)

In [584]:
# Detrend title and body
detrended_title = []
for i, val in enumerate(corpus_lem_title):
    if sentiment_title[i]['compound'] >= 0.10:
        detrended_title.append(title_2_vec[i] - get_w2vec('great'))
    elif sentiment_title[i]['compound'] <= 0.10:
        detrended_title.append(title_2_vec[i] - get_w2vec('hate'))
    else:
        detrended_title.append(title_2_vec[i])
        
detrended_body = []
for i, val in enumerate(corpus_lem_body):
    if sentiment_body[i]['compound'] >= 0.10:
        detrended_body.append(body_2_vec[i] - get_w2vec('great'))
    elif sentiment_body[i]['compound'] <= 0.10:
        detrended_body.append(body_2_vec[i] - get_w2vec('hate'))
    else:
        detrended_body.append(body_2_vec[i])

In [656]:
# Get sentiment free w2vecs

input_str = 'Stocks are the thing I love'
sentiment_input = analyzer.polarity_scores(input_str)

if sentiment_input['compound'] >= 0.10:
    detrend_input_vec = get_w2vec(input_str) - get_w2vec('love')
elif sentiment_input['compound'] <= 0.10:
    detrend_input_vec = get_w2vec(input_str) - get_w2vec('hate')
else:
    detrend_input_vec = get_w2vec(input_str)

In [665]:
def search_results(input_vec, search_over_vec, print_from_list, num):
    cos_dist_df = get_closest_stuff_detrended(input_vec, search_over_vec)
    print_posts(cos_dist_df, print_from_list, num=num)
    
def biased_search_title(input_vec, bias, num):
    if bias == 0:
        return search_results(detrend_input_vec, detrended_title, grabbed_list_title, num)
    elif bias > 0:
        return search_results(detrend_input_vec+get_w2vec('love'), title_2_vec, grabbed_list_title, num)
    elif bias < 0:
        return search_results(detrend_input_vec+get_w2vec('hate'), title_2_vec, grabbed_list_title, num)
    
def biased_search_body(input_vec, bias, num):
    if bias == 0:
        return search_results(detrend_input_vec, detrended_body, grabbed_list_body, num)
    elif bias > 0:
        return search_results(detrend_input_vec+get_w2vec('love'), body_2_vec, grabbed_list_body, num)
    elif bias < 0:
        return search_results(detrend_input_vec+get_w2vec('hate'), body_2_vec, grabbed_list_body, num)

In [670]:
biased_search_title(get_w2vec(input_str), bias=0, num=5)

Stocks are down
Stock wishlist?
AQ stock buyout
BYND Stock
KHC Stock
